In [1]:
import math
import gc
import pickle
import random
from copy import deepcopy
from tqdm import tqdm
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
import optuna
from optuna.samplers import RandomSampler, TPESampler, GPSampler
import warnings
warnings.filterwarnings("ignore")
# import multiprocessing
# max_n_jobs = multiprocessing.cpu_count()
import shap
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader, Sampler
import torch.nn as nn
import torch.optim as optim
import mlx.core as mx
import mlx.nn as nnmx
import mlx.optimizers as optimmx

/Users/justpqa/drw-crypto-market-prediction/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set the device
device = "mps" if torch.backends.mps.is_available() else "cpu"

In [3]:
feature_version = 2
# 1 for pc feature, 
# 2 for label correlation feature # seems to work most consistently
# 3 for best features based on combination rank
# 4 for including time features (in case we want to reverse engineer the masked timestamp)
# 5 for increasing number of correlation features + only use those that are in the same cluster
# 6 is for 2 but more features, use when I want to use more features for larger models or AE approaches

In [4]:
default_random_state = 101
random.seed(default_random_state)
np.random.seed(default_random_state)
torch.manual_seed(default_random_state)
torch.mps.manual_seed(default_random_state)
mx.random.seed(default_random_state)

#### Import train data and popular features

In [5]:
train_df = pd.read_parquet(f"data/cleaned/cleaned_train_{feature_version}.parquet")
train_df.head()

,X751,X473,X472,X451,X226,X219,X205,X445,X444,X27,...,X758_X219_X89_interaction,X508_X465_X226_interaction,X508_X466_X226_interaction,X445_X466_X226_interaction,X465_X466_X226_interaction,X466_X226_X444_interaction,X95_X169_X272_interaction,X89_X169_X272_interaction,__index_level_0__,label
0,0.000617,0.362816,0.255354,0.625153,-0.893206,-0.654146,-1.250753,0.755891,0.625328,1.714323,...,0.356342,-0.060688,-0.088965,-0.232081,-0.071993,-0.191994,0.432966,0.495033,2023-03-01 00:00:00,0.562539
1,0.013388,0.378391,0.274621,0.637250,-0.738291,-0.634723,-1.100357,0.760472,0.633046,1.396133,...,0.342402,-0.051643,-0.074490,-0.195497,-0.062058,-0.162739,0.460443,0.525698,2023-03-01 00:01:00,0.533686
2,-0.016807,0.382337,0.279272,0.640437,-0.713420,-0.631882,-1.073226,0.761631,0.635009,1.205921,...,0.339868,-0.048644,-0.069951,-0.189648,-0.060436,-0.158119,0.468474,0.534105,2023-03-01 00:02:00,0.546505
3,-0.036622,0.387473,0.284750,0.642831,-0.644172,-0.612901,-0.982398,0.761936,0.635508,1.419536,...,0.328690,-0.042121,-0.060503,-0.171468,-0.054733,-0.143016,0.461758,0.525697,2023-03-01 00:03:00,0.357703
4,-0.053322,0.390820,0.289431,0.648175,-0.628840,-0.607648,-0.952145,0.764770,0.640311,1.408936,...,0.324915,-0.039559,-0.056454,-0.168882,-0.054339,-0.141398,0.450168,0.511770,2023-03-01 00:04:00,0.362452


In [6]:
popular_features_train = pd.read_parquet("data/cleaned/popular_features_train.parquet")
popular_features_train.head()

,volume,bid_qty,ask_qty,buy_qty,sell_qty
0,221.389,15.283,8.425,176.405,44.984
1,847.796,38.590,2.336,525.846,321.950
2,295.596,0.442,60.250,159.227,136.369
3,460.705,4.865,21.016,335.742,124.963
4,142.818,27.158,3.451,98.411,44.407


#### Implement some helper function

In [7]:
# First need to split into some fold
train_df["__index_level_0__"] = pd.to_datetime(train_df["__index_level_0__"])

default_cv = 4
default_cv_type = "full"
# NOTE: default_cv must set to 1 instead of 3 based on consistency with LB score contains 49% of test data
# NOTE: 3 cv with gap is slightly better or almost equal

def create_cv(train_df, features=None, cv=default_cv):
    if features is not None:
        train_df = train_df[features + ["__index_level_0__", "label"]]
    X_train_arr = []
    X_test_arr = []
    Y_train_arr = []
    Y_test_arr = []
    for i in range(cv):
        # if i == 0:
        #     train_month = [3, 4, 5, 6, 7, 8]
        #     test_month = [9, 10, 11, 12, 1, 2]
        # else:
        train_month = list(range(3 + i, 7 + i))
        test_month = list(map(lambda x: x % 12 if x > 12 else x, list(range(8 + i, 12 + i))))
        print(train_month, test_month)
        # print(train_month, test_month)
        train = train_df[train_df["__index_level_0__"].dt.month.isin(train_month)].reset_index().drop("index", axis = 1)
        test = train_df[train_df["__index_level_0__"].dt.month.isin(test_month)].reset_index().drop("index", axis = 1)
        X_train_arr.append(train.drop(["__index_level_0__", "label"], axis = 1))
        X_test_arr.append(test.drop(["__index_level_0__", "label"], axis = 1))
        Y_train_arr.append(train["label"])
        Y_test_arr.append(test["label"])  
    return X_train_arr, X_test_arr, Y_train_arr, Y_test_arr

# def create_cv_random_test(train_df, features=None, test_cv=10):
#     # randomize so that we have 1 train, but try it on 10 different test 
#     if features is not None:
#         train_df = train_df[features + ["timestamp", "label"]]
#     X_train_arr = []
#     X_test_arr = []
#     Y_train_arr = []
#     Y_test_arr = []

#     # Create train data
#     train_month = [3, 4, 5, 6, 7, 8]
#     train = train_df[train_df["timestamp"].dt.month.isin(train_month)] 
#     X_train_arr.append(train.drop(["timestamp", "label"], axis = 1))
#     Y_train_arr.append(train["label"])

#     test_month = [9, 10, 11, 12, 1, 2]
#     test = train_df[train_df["timestamp"].dt.month.isin(test_month)]
#     # Create test data
#     for _ in range(test_cv):
#         random_test = test.sample(frac = 0.5, random_state = default_random_state)
#         X_test_arr.append(random_test.drop(["timestamp", "label"], axis = 1))
#         Y_test_arr.append(random_test["label"])

#     return X_train_arr, X_test_arr, Y_train_arr, Y_test_arr 

# class [-1, 0, 1] -> [0, 1, 2] => < -0.2 => neg, > 0.2 => pos, else => neutral
def create_classification_class(label):
    if label < -0.4: return 0
    elif label < 0: return 1
    elif label < 0.4: return 2
    return 3

def create_cv_classification(train_df, features=None, cv=default_cv):
    if features is not None:
        train_df = train_df[features + ["__index_level_0__", "label"]]
    X_train_arr = []
    X_test_arr = []
    Y_train_arr = []
    Y_test_arr = []
    for i in range(cv):
        train_month = list(range(3 + i, 7 + i))
        # train_month = [3, 4, 5, 6, 7, 8]
        test_month = list(map(lambda x: x % 12 if x > 12 else x, list(range(8 + i, 12 + i))))
        print(train_month, test_month)
        # test_month = [9, 10, 11, 12, 1, 2] # try to make a gap to see if there is any differences in cv-lb correlation
        # print(train_month, test_month)
        train = train_df[train_df["__index_level_0__"].dt.month.isin(train_month)].reset_index().drop("index", axis = 1)
        test = train_df[train_df["__index_level_0__"].dt.month.isin(test_month)].reset_index().drop("index", axis = 1)
        X_train_arr.append(train.drop(["__index_level_0__", "label"], axis = 1))
        X_test_arr.append(test.drop(["__index_level_0__", "label"], axis = 1))
        Y_train_arr.append(train["label"].apply(lambda x: create_classification_class(x)))
        Y_test_arr.append(test["label"].apply(lambda x: create_classification_class(x)))  
    return X_train_arr, X_test_arr, Y_train_arr, Y_test_arr

In [8]:
def pearson_score(Y_test, Y_pred):
    if isinstance(Y_test, pd.Series) or isinstance(Y_test, pd.DataFrame):
        Y_test = Y_test.values
    if isinstance(Y_pred, pd.Series) or isinstance(Y_pred, pd.DataFrame):
        Y_pred = Y_pred.values
    Y_test = np.ravel(Y_test)
    Y_pred = np.ravel(Y_pred)
    pearson = np.corrcoef(Y_test, Y_pred)[0, 1]
    if np.isnan(pearson):
        if np.std(Y_pred) == 0:
            print(Y_pred)
            print("Error: zero variance prediction")
        elif np.isnan(Y_pred).any():
            print("Error: nan prediction")
        return -1
    else:
        return pearson

In [9]:
# Make function specifically for cross validation
def train_eval_cv(model, cv, X_train_arr, X_test_arr, Y_train_arr, Y_test_arr, scoring_function=pearson_score):
    cv_score = 0

    for i in range(cv):
        X_train, X_test = X_train_arr[i], X_test_arr[i]
        Y_train, Y_test = Y_train_arr[i], Y_test_arr[i]
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        cv_score += scoring_function(Y_test, Y_pred)
    
    return cv_score / cv

def train_eval_cv_random_test(model, cv, X_train_arr, X_test_arr, Y_train_arr, Y_test_arr, scoring_function=pearson_score, test_cv = 10):
    cv_score = 0

    for i in range(cv):
        curr_cv_score = 0

        # Conduct fitting
        X_train, X_test = X_train_arr[i], X_test_arr[i]
        Y_train, Y_test = Y_train_arr[i], Y_test_arr[i]
        model.fit(X_train, Y_train)
        
        # sampling and testing
        len_test = X_test.shape[0]
        for seed in tqdm(range(test_cv)):
            np.random.seed(seed)
            test_index = np.random.choice(len_test, size = len_test // 2, replace = False) 
            X_test_sample = X_test.loc[test_index, :]
            Y_test_sample = Y_test[test_index]
            Y_pred_sample = model.predict(X_test_sample)
            curr_cv_score += scoring_function(Y_test_sample, Y_pred_sample)
        
        cv_score += curr_cv_score / test_cv
    
    np.random.seed(default_random_state)
    return cv_score

In [10]:
default_n_trees = 1000
# Finetuning XGBoost
def objective_xgboost(trial):
    params = {
        "n_estimators": default_n_trees,
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log = True), # 0.001 - 0.1 -> 0.01 - 0.05 
        "verbosity": 0,
        "subsample": trial.suggest_float("subsample", 0.05, 1.0), # 1.0 -> 0.2
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1),
        "colsample_bynode": trial.suggest_float("colsample_bynode", 0.05, 1), 
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1), 
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 100),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 100),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "enable_categorical": True,
        "random_state": default_random_state
    }

    xgbr = XGBRegressor(**params)
    cv_pearson = train_eval_cv(xgbr, default_cv, X_train_arr, X_test_arr, Y_train_arr, Y_test_arr, pearson_score)
    return cv_pearson

def objective_lightgbm(trial):
    params = {
        "n_estimators": default_n_trees,
        "verbosity": -1,
        "max_depth": trial.suggest_int("max_depth", 2, 10), # 1 - 10 => 1 - 5
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True), # 0.001 - 0.1 -> 0.005 - 0.02
        "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_child_weight": trial.suggest_float("min_child_weight", 0, 1),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 100),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 100),
        "random_state": default_random_state
    }

    lgbr = LGBMRegressor(**params)
    cv_pearson = train_eval_cv(lgbr, default_cv, X_train_arr, X_test_arr, Y_train_arr, Y_test_arr, pearson_score)
    return cv_pearson

def objective_catboost(trial):
    params = {
        "iterations": default_n_trees,
        "verbose": False,
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True), # 0.001 - 0.1 => 0.01 - 0.1
        "depth": trial.suggest_int("depth", 1, 10), #  1 - 10 => 5 - 15
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 2, 600),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 100),
        "random_seed": default_random_state
    }

    cbr = CatBoostRegressor(**params)
    cv_pearson = train_eval_cv(cbr, default_cv, X_train_arr, X_test_arr, Y_train_arr, Y_test_arr, pearson_score)
    return cv_pearson

In [11]:
# Finetuning XGBoost
def objective_xgboost_classification(trial):
    params = {
        "n_estimators": default_n_trees,
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log = True), # 0.001 - 0.1 -> 0.01 - 0.05 
        "verbosity": 0,
        "subsample": trial.suggest_float("subsample", 0.05, 1.0), # 1.0 -> 0.2
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1),
        "colsample_bynode": trial.suggest_float("colsample_bynode", 0.05, 1), 
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1), 
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 100),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 100),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "enable_categorical": True,
        "random_state": default_random_state
    }

    xgbr = XGBClassifier(**params)
    cv_acc = train_eval_cv(xgbr, default_cv, X_train_arr, X_test_arr, Y_train_arr, Y_test_arr, accuracy_score)
    return cv_acc

def objective_lightgbm_classification(trial):
    params = {
        "n_estimators": default_n_trees,
        "verbosity": -1,
        "max_depth": trial.suggest_int("max_depth", 2, 10), # 1 - 10 => 1 - 5
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True), # 0.001 - 0.1 -> 0.005 - 0.02
        "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_child_weight": trial.suggest_float("min_child_weight", 0, 1),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 100),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 100),
        "random_state": default_random_state
    }

    lgbr = LGBMClassifier(**params)
    cv_acc = train_eval_cv(lgbr, default_cv, X_train_arr, X_test_arr, Y_train_arr, Y_test_arr, accuracy_score)
    return cv_acc

def objective_catboost_classification(trial):
    params = {
        "iterations": default_n_trees,
        "verbose": False,
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True), # 0.001 - 0.1 => 0.01 - 0.1
        "depth": trial.suggest_int("depth", 1, 10), #  1 - 10 => 5 - 15
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 2, 600),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 100),
        "random_seed": default_random_state
    }

    cbr = CatBoostRegressor(**params)
    cv_acc = train_eval_cv(cbr, default_cv, X_train_arr, X_test_arr, Y_train_arr, Y_test_arr, accuracy_score)
    return cv_acc

In [12]:
default_n_trials = 100
default_n_jobs = 1

def optimize_xgboost(study_name, storage_name, objective_function=objective_xgboost, n_trials = default_n_trials, n_jobs = default_n_jobs):
    print("Conduct hyperparam opt for XGBoost")
    study = optuna.create_study(
        study_name = study_name,
        direction ='maximize',
        storage = f"sqlite:///{storage_name}.db",
        sampler = TPESampler(seed = 101, n_startup_trials=10),
        load_if_exists=True
    )
    study.optimize(objective_function, n_trials=n_trials, n_jobs=n_jobs)
    print('Best hyperparameters:', study.best_params)
    print('Best Pearson score:', study.best_value)
    return study.best_params

def optimize_lightgbm(study_name, storage_name, objective_function=objective_lightgbm, n_trials = default_n_trials, n_jobs = default_n_jobs):
    print("Conduct hyperparam opt for LightGBM")
    study = optuna.create_study(
        study_name = study_name,
        direction='maximize',
        storage = f"sqlite:///{storage_name}.db",
        sampler = TPESampler(seed = 101, n_startup_trials=10),
        load_if_exists=True
    )
    study.optimize(objective_function, n_trials=n_trials, n_jobs=n_jobs)
    print('Best hyperparameters:', study.best_params)
    print('Best Pearson score:', study.best_value)
    return study.best_params

def optimize_catboost(study_name, storage_name, objective_function=objective_catboost, n_trials = default_n_trials, n_jobs = default_n_jobs):
    print("Conduct hyperparam opt for CatBoost")
    study = optuna.create_study(
        study_name = study_name,
        direction='maximize',
        storage = f"sqlite:///{storage_name}.db",
        sampler = TPESampler(seed = 101, n_startup_trials=10),
        load_if_exists=True
    )
    study.optimize(objective_function, n_trials=n_trials, n_jobs=n_jobs)
    print('Best hyperparameters:', study.best_params)
    print('Best Pearson score:', study.best_value)
    return study.best_params

#### First iteration: training with all features from the collection, no popular features

In [ ]:
original_features = [f for f in train_df.columns if "X" in f]

X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv(train_df, original_features)

In [ ]:
best_params_xgboost = optimize_xgboost(
    f"xgboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_study",
    f"xgboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_study"
)

In [ ]:
best_params_lightgbm = optimize_lightgbm(
    f"lightgbm_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_study",
    f"lightgbm_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_study"
)

In [ ]:
# best_params_catboost = optimize_catboost(
#     f"catboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_study",
#     f"catboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_study"
# )
# # Need to take down as catboost might not work well in this situation

Analyze params - cv relationship

In [13]:
def get_study_df(filename):
    study = optuna.load_study(
        study_name = filename,
        storage = f"sqlite:///{filename}.db"
    )
    study_df = []
    for trial in study.trials:
        trial_dict = trial.params
        trial_dict["value"] = trial.value
        study_df.append(trial_dict)

    return pd.DataFrame(study_df)

In [14]:
def params_value_viz(study_df):
    nrows = (study_df.shape[1] - 1) // 3 + ((study_df.shape[1] - 1) % 3 > 0)
    fig, ax = plt.subplots(nrows = nrows, ncols = 3, figsize = (14, 5 * nrows))
    for inx, var in enumerate(study_df.columns):
        x, y = inx // 3, inx % 3
        if var != "value":
            sns.regplot(study_df, x = var, y = "value", ax = ax[x][y], lowess=True, line_kws={'color': 'green'}, ci = 95)
    plt.show()

In [ ]:
study_df_xgboost = get_study_df(f"xgboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_study")   
params_value_viz(study_df_xgboost)

In [ ]:
study_df_lightgbm = get_study_df(f"lightgbm_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_study")
params_value_viz(study_df_lightgbm)

In [ ]:
# study_df_catboost = get_study_df(f"catboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_study")
# params_value_viz(study_df_catboost)

Analyze feature importance + CV performance

In [15]:
def get_best_params_from_file(filename):
    study = optuna.load_study(
        study_name = filename,
        storage = f"sqlite:///{filename}.db"
    )
    return study.best_params

In [16]:
def get_shap_values(model, X_train_arr, X_test_arr, Y_train_arr, Y_test_arr, sample_size=10000):
    mean_abs_shap_all = np.zeros(X_train_arr[0].shape[1])
    for i in range(default_cv):
        X_train, X_test = X_train_arr[i], X_test_arr[i]
        Y_train, Y_test = Y_train_arr[i], Y_test_arr[i]
        model.fit(X_train, Y_train)
        X_test_sample = X_test.sample(sample_size, random_state = default_random_state)
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_test)
        mean_abs_shap = np.mean(np.abs(shap_values), axis = 0)
        mean_abs_shap_all += mean_abs_shap
    mean_abs_shap_all /= default_cv
    return mean_abs_shap_all

In [ ]:
params = {
    "n_estimators": default_n_trees,
    "verbosity": 0,
    "enable_categorical": True,
    "random_state": default_random_state
}
best_params_xgboost = get_best_params_from_file(f"xgboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_study")
for p in best_params_xgboost:
    params[p] = best_params_xgboost[p]

xgboost_feature_importances = {}

xgbr = XGBRegressor(**params)
for i in range(default_cv):
    X_train, X_test = X_train_arr[i], X_test_arr[i]
    Y_train, Y_test = Y_train_arr[i], Y_test_arr[i]
    xgbr.fit(X_train, Y_train)
    print(pearson_score(Y_test, xgbr.predict(X_test)))
    features = xgbr.feature_names_in_.tolist()
    # features_i = xgbr.feature_importances_.tolist()
    features_i = get_shap_values(xgbr, X_train_arr, X_test_arr, Y_train_arr, Y_test_arr)
    for inx, feat in enumerate(features):
        xgboost_feature_importances[feat] = xgboost_feature_importances.get(feat, 0) + features_i[inx]

# print(feature_importances)
plt.hist(xgboost_feature_importances.values())
# Seems like only COD features are important (can try to only use 4-8 hours if 4-13 hours does not work well)

In [ ]:
print([f for f in xgboost_feature_importances if xgboost_feature_importances[f] > 0.01])

In [ ]:
params = {
    "n_estimators": default_n_trees,
    "verbosity": -1,
    "random_state": default_random_state
}
best_params_lightgbm = get_best_params_from_file(f"lightgbm_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_study")
for p in best_params_lightgbm:
    params[p] = best_params_lightgbm[p]

lightgbm_feature_importances = {}

lgbr = LGBMRegressor(**params)
for i in range(default_cv):
    X_train, X_test = X_train_arr[i], X_test_arr[i]
    Y_train, Y_test = Y_train_arr[i], Y_test_arr[i]
    lgbr.fit(X_train, Y_train)
    print(pearson_score(Y_test, lgbr.predict(X_test)))
    features = lgbr.feature_names_in_.tolist()
    # features_i = lgbr.feature_importances_.tolist()
    features_i = get_shap_values(lgbr, X_train_arr, X_test_arr, Y_train_arr, Y_test_arr)
    for inx, feat in enumerate(features):
        lightgbm_feature_importances[feat] = lightgbm_feature_importances.get(feat, 0) + features_i[inx]

plt.hist(lightgbm_feature_importances.values())
# seems to pick up time features not as good as past 4 hours features

In [ ]:
print([f for f in lightgbm_feature_importances if lightgbm_feature_importances[f] >= 0.01])

In [ ]:
# params = {
#     "iterations": default_n_trees,
#     "verbose": False,
#     "random_seed": default_random_state
# }
# best_params_catboost = get_best_params_from_file(f"catboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_study")
# for p in best_params_catboost:
#     params[p] = best_params_catboost[p]

# catboost_feature_importances = {}

# cbr = CatBoostRegressor(**params)
# cv_rmse = 0

# for i in range(default_cv):
#     X_train, X_test = X_train_arr[i], X_test_arr[i]
#     Y_train, Y_test = Y_train_arr[i], Y_test_arr[i]
#     cbr.fit(X_train, Y_train)
#     print(pearson_score(Y_test, cbr.predict(X_test)))
#     features = cbr.feature_names_
#     # features_i = cbr.feature_importances_.tolist()
#     features_i = get_shap_values(cbr, X_train_arr, X_test_arr, Y_train_arr, Y_test_arr)
#     for inx, feat in enumerate(features):
#         catboost_feature_importances[feat] = catboost_feature_importances.get(feat, 0) + features_i[inx]

# plt.hist(catboost_feature_importances.values())
# # can pick up a combination of both past cod and tss, not good at picking up ph, temp

In [ ]:
# print([f for f in catboost_feature_importances if catboost_feature_importances[f] >= 0.02])

Get top 20 important features in all of them

In [ ]:
xgboost_feature_importances_df = pd.DataFrame(
    {"var": xgboost_feature_importances.keys(), "importance": xgboost_feature_importances.values()}
)
xgboost_feature_importances_df["importance"] /= default_cv
# xgboost_feature_importances_df["rank_importance"] = xgboost_feature_importances_df["importance"].rank(ascending=False)
lightgbm_feature_importances_df = pd.DataFrame(
    {"var": lightgbm_feature_importances.keys(), "importance": lightgbm_feature_importances.values()}
)
lightgbm_feature_importances_df["importance"] /= default_cv
# lightgbm_feature_importances_df["rank_importance"] = lightgbm_feature_importances_df["importance"].rank(ascending=False)
# catboost_feature_importances_df = pd.DataFrame(
#     {"var": catboost_feature_importances.keys(), "importance_catboost": catboost_feature_importances.values()}
# )
# catboost_feature_importances_df["rank_importance"] = catboost_feature_importances_df["importance_catboost"].rank(ascending=False)
feature_importances_df = xgboost_feature_importances_df.merge(
    lightgbm_feature_importances_df,
    on="var",
    how="inner",
    suffixes=("_xgboost", "_lightgbm")
)
# feature_importances_df = feature_importances_df.merge(
#     catboost_feature_importances_df,
#     on="var",
#     how="inner",
#     suffixes=("", "_catboost")
# )
# feature_importances_df = feature_importances_df[["var", "rank_importance_xgboost", "rank_importance_lightgbm", "rank_importance_catboost"]]
# feature_importances_df["rank"] = 1/3 * (feature_importances_df["rank_importance_xgboost"] + feature_importances_df["rank_importance_lightgbm"] + feature_importances_df["rank_importance_catboost"])

feature_importances_df["importance"] = 1/2 * (feature_importances_df["importance_xgboost"] + feature_importances_df["importance_lightgbm"])
feature_importances_df = feature_importances_df.sort_values(by="importance", ascending=False).reset_index().drop("index", axis = 1)
feature_importances_df[:50]

In [ ]:
feature_importances_df.to_csv("feature_importances_df.csv", index = False)

In [ ]:
feature_importances_df = pd.read_csv("feature_importances_df.csv")
feature_importances_df

In [ ]:
print(feature_importances_df.loc[:49, "var"].tolist())

#### Second Iteration: adding popular feature in addition to original features correlated to label

In [ ]:
train_added_df = pd.concat([train_df, popular_features_train], axis = 1)

X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv(train_added_df)

In [ ]:
best_params_xgboost_popular_feature = optimize_xgboost(
    f"xgboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_popular_feature_study",
    f"xgboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_popular_feature_study"
)

In [ ]:
best_params_lightgbm_popular_feature = optimize_lightgbm(
    f"lightgbm_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_popular_feature_study",
    f"lightgbm_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_popular_feature_study"
)

Check for feature importance

In [ ]:
params = {
    "n_estimators": default_n_trees,
    "verbosity": 0,
    "enable_categorical": True,
    "random_state": default_random_state
}
best_params_xgboost_popular_feature = get_best_params_from_file(f"xgboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_popular_feature_study")
for p in best_params_xgboost_popular_feature:
    params[p] = best_params_xgboost_popular_feature[p]

xgboost_feature_importances = {}

xgbr = XGBRegressor(**params)
for i in range(default_cv):
    X_train, X_test = X_train_arr[i], X_test_arr[i]
    Y_train, Y_test = Y_train_arr[i], Y_test_arr[i]
    xgbr.fit(X_train, Y_train)
    print(pearson_score(Y_test, xgbr.predict(X_test)))
    features = xgbr.feature_names_in_.tolist()
    # features_i = xgbr.feature_importances_.tolist()
    features_i = get_shap_values(xgbr, X_train_arr, X_test_arr, Y_train_arr, Y_test_arr)
    for inx, feat in enumerate(features):
        xgboost_feature_importances[feat] = xgboost_feature_importances.get(feat, 0) + features_i[inx]

# print(feature_importances)
plt.hist(xgboost_feature_importances.values())
# Seems like only COD features are important (can try to only use 4-8 hours if 4-13 hours does not work well)

In [ ]:
params = {
    "n_estimators": default_n_trees,
    "verbosity": -1,
    "random_state": default_random_state
}
best_params_lightgbm_popular_feature = get_best_params_from_file(f"lightgbm_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_popular_feature_study")
for p in best_params_lightgbm_popular_feature:
    params[p] = best_params_lightgbm_popular_feature[p]

lightgbm_feature_importances = {}

lgbr = LGBMRegressor(**params)
for i in range(default_cv):
    X_train, X_test = X_train_arr[i], X_test_arr[i]
    Y_train, Y_test = Y_train_arr[i], Y_test_arr[i]
    lgbr.fit(X_train, Y_train)
    print(pearson_score(Y_test, lgbr.predict(X_test)))
    features = lgbr.feature_names_in_.tolist()
    # features_i = lgbr.feature_importances_.tolist()
    features_i = get_shap_values(lgbr, X_train_arr, X_test_arr, Y_train_arr, Y_test_arr)
    for inx, feat in enumerate(features):
        lightgbm_feature_importances[feat] = lightgbm_feature_importances.get(feat, 0) + features_i[inx]

plt.hist(lightgbm_feature_importances.values())
# seems to pick up time features not as good as past 4 hours features

In [ ]:
xgboost_feature_importances_df = pd.DataFrame(
    {"var": xgboost_feature_importances.keys(), "importance": xgboost_feature_importances.values()}
)
xgboost_feature_importances_df["importance"] /= default_cv
# xgboost_feature_importances_df["rank_importance"] = xgboost_feature_importances_df["importance"].rank(ascending=False)
lightgbm_feature_importances_df = pd.DataFrame(
    {"var": lightgbm_feature_importances.keys(), "importance": lightgbm_feature_importances.values()}
)
lightgbm_feature_importances_df["importance"] /= default_cv
# lightgbm_feature_importances_df["rank_importance"] = lightgbm_feature_importances_df["importance"].rank(ascending=False)
# catboost_feature_importances_df = pd.DataFrame(
#     {"var": catboost_feature_importances.keys(), "importance_catboost": catboost_feature_importances.values()}
# )
# catboost_feature_importances_df["rank_importance"] = catboost_feature_importances_df["importance_catboost"].rank(ascending=False)
feature_importances_df = xgboost_feature_importances_df.merge(
    lightgbm_feature_importances_df,
    on="var",
    how="inner",
    suffixes=("_xgboost", "_lightgbm")
)
# feature_importances_df = feature_importances_df.merge(
#     catboost_feature_importances_df,
#     on="var",
#     how="inner",
#     suffixes=("", "_catboost")
# )
# feature_importances_df = feature_importances_df[["var", "rank_importance_xgboost", "rank_importance_lightgbm", "rank_importance_catboost"]]
# feature_importances_df["rank"] = 1/3 * (feature_importances_df["rank_importance_xgboost"] + feature_importances_df["rank_importance_lightgbm"] + feature_importances_df["rank_importance_catboost"])
feature_importances_df["importance"] = 1/2 * (feature_importances_df["importance_xgboost"] + feature_importances_df["importance_lightgbm"])
feature_importances_df = feature_importances_df.sort_values(by="importance", ascending=False).reset_index().drop("index", axis = 1)
feature_importances_df

In [ ]:
best_xgboost_score = optuna.load_study(
    study_name = "xgboost_2_4_101_1000_popular_feature_study",
    storage = f"sqlite:///xgboost_2_4_101_1000_popular_feature_study.db"
).best_value
best_lightgbm_score = optuna.load_study(
    study_name = "lightgbm_2_4_101_1000_popular_feature_study",
    storage = f"sqlite:///lightgbm_2_4_101_1000_popular_feature_study.db"
).best_value
feature_importances_df["weighted_importance"] = (best_xgboost_score * feature_importances_df["importance_xgboost"] + best_lightgbm_score * feature_importances_df["importance_lightgbm"]) / (best_xgboost_score + best_lightgbm_score)
feature_importances_df = feature_importances_df.sort_values("weighted_importance", ascending=False, ignore_index=True)
feature_importances_df

In [ ]:
feature_importances_df.to_csv("feature_importances_df.csv", index = False)

In [ ]:
feature_importances_df = pd.read_csv("feature_importances_df.csv")
feature_importances_df

In [ ]:
feature_importances_df[~feature_importances_df["var"].str.contains("X")]

In [ ]:
print(feature_importances_df.sort_values("importance", ignore_index=True, ascending=False).head(30)["var"].tolist())

In [ ]:
print(feature_importances_df.sort_values("weighted_importance", ignore_index=True, ascending=False).head(30)["var"].tolist())

In [ ]:
s1 = set(feature_importances_df.sort_values("importance", ignore_index=True, ascending=False).head(20)["var"].tolist())
s2 = set(feature_importances_df.sort_values("weighted_importance", ignore_index=True, ascending=False).head(20)["var"].tolist())
print(s1 - s2)
print(s2 - s1)

In [ ]:
feature_importances_df.sort_values("importance", ignore_index=True, ascending=False).head(50)

In [ ]:
feature_importances_df.sort_values("weighted_importance", ignore_index=True, ascending=False).head(50)

#### Third Iteration: a common truncated version using good features across all models + popular features

In [ ]:
best_features = ['X757', 'X758', 'X759', 'X508', 'X614', 'X752', 'X331', 'X445', 'X465', 'X385', 
                 'X466', 'X95', 'X23', 'X219', 'X31', 'X373', 'X379', 'X284', 'X750', 'X652', 
                 'X279', 'X89', 'X169', 'X753', 'X226', 'X28', 'X444', 'X272', 'X271', 'X218']
# Best is at 30 features with no popular features
best_features = list(set(best_features))
train_added_df = pd.concat([train_df, popular_features_train], axis=1)
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv(train_added_df, best_features)

XGBoost

In [ ]:
best_xgboost_params_common_truncated = optimize_xgboost(
    f"xgboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_common_truncated_{len(best_features)}_study",
    f"xgboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_common_truncated_{len(best_features)}_study"
) 

LightGBM

In [ ]:
best_lightgbm_params_common_truncated = optimize_lightgbm(
    f"lightgbm_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_common_truncated_{len(best_features)}_study",
    f"lightgbm_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_common_truncated_{len(best_features)}_study"
)

Catboost

In [ ]:
# best_catboost_params_common_truncated = optimize_catboost(
#     f"catboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_common_truncated_20_study",
#     f"catboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_common_truncated_20_study"
# )

Analyze model performance and feature importance across train and test

In [ ]:
params = {
    "n_estimators": default_n_trees,
    "verbosity": 0,
    "enable_categorical": True,
    "random_state": default_random_state
}
best_params_xgboost = get_best_params_from_file(f"xgboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_common_truncated_20_study")
for p in best_params_xgboost:
    params[p] = best_params_xgboost[p]

xgbr_arr = []

for i in tqdm(range(default_cv)):
    xgbr = XGBRegressor(**params)
    xgbr.fit(X_train_arr[i], Y_train_arr[i])
    xgbr_arr.append(xgbr)

In [ ]:
params = {
    "n_estimators": default_n_trees,
    "verbosity": -1,
    "random_state": default_random_state,
}
best_params_lightgbm = get_best_params_from_file(f"lightgbm_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_common_truncated_20_study")
for p in best_params_lightgbm:
    params[p] = best_params_lightgbm[p]

lgbr_arr = []

for i in tqdm(range(default_cv)):
    lgbr = LGBMRegressor(**params)
    lgbr.fit(X_train_arr[i], Y_train_arr[i])
    lgbr_arr.append(lgbr)

In [ ]:
xgboost_feature_importances = {}
lightgbm_feature_importances = {}

for i in tqdm(range(default_cv)):
    features = xgbr_arr[i].feature_names_in_.tolist()
    features_i = get_shap_values(xgbr_arr[i], X_train_arr, X_test_arr, Y_train_arr, Y_test_arr)
    for inx, feat in enumerate(features):
        xgboost_feature_importances[feat] = xgboost_feature_importances.get(feat, 0) + features_i[inx]
    features = lgbr_arr[i].feature_names_in_.tolist()
    features_i = get_shap_values(xgbr_arr[i], X_train_arr, X_test_arr, Y_train_arr, Y_test_arr)
    for inx, feat in enumerate(features):
        lightgbm_feature_importances[feat] = lightgbm_feature_importances.get(feat, 0) + features_i[inx]

xgboost_feature_importances_df = pd.DataFrame(
    {"var": xgboost_feature_importances.keys(), "importance": xgboost_feature_importances.values()}
)
xgboost_feature_importances_df["importance"] /= default_cv
# xgboost_feature_importances_df["rank_importance"] = xgboost_feature_importances_df["importance"].rank(ascending=False)
lightgbm_feature_importances_df = pd.DataFrame(
    {"var": lightgbm_feature_importances.keys(), "importance": lightgbm_feature_importances.values()}
)
lightgbm_feature_importances_df["importance"] /= default_cv
# lightgbm_feature_importances_df["rank_importance"] = lightgbm_feature_importances_df["importance"].rank(ascending=False)
# catboost_feature_importances_df = pd.DataFrame(
#     {"var": catboost_feature_importances.keys(), "importance_catboost": catboost_feature_importances.values()}
# )
# catboost_feature_importances_df["rank_importance"] = catboost_feature_importances_df["importance_catboost"].rank(ascending=False)
feature_importances_df_common_truncated = xgboost_feature_importances_df.merge(
    lightgbm_feature_importances_df,
    on="var",
    how="inner",
    suffixes=("_xgboost", "_lightgbm")
)
# feature_importances_df = feature_importances_df.merge(
#     catboost_feature_importances_df,
#     on="var",
#     how="inner",
#     suffixes=("", "_catboost")
# )
# feature_importances_df = feature_importances_df[["var", "rank_importance_xgboost", "rank_importance_lightgbm", "rank_importance_catboost"]]
# feature_importances_df["rank"] = 1/3 * (feature_importances_df["rank_importance_xgboost"] + feature_importances_df["rank_importance_lightgbm"] + feature_importances_df["rank_importance_catboost"])
feature_importances_df_common_truncated["importance"] = 1/2 * (feature_importances_df_common_truncated["importance_xgboost"] + feature_importances_df_common_truncated["importance_lightgbm"])
feature_importances_df_common_truncated = feature_importances_df_common_truncated.sort_values(by="importance", ascending=False).reset_index().drop("index", axis = 1)
feature_importances_df_common_truncated

In [ ]:
best_xgboost_score = optuna.load_study(
    study_name = "xgboost_2_4_101_1000_common_truncated_20_study",
    storage = f"sqlite:///xgboost_2_4_101_1000_common_truncated_20_study.db"
).best_value
best_lightgbm_score = optuna.load_study(
    study_name = "lightgbm_2_4_101_1000_common_truncated_20_study",
    storage = f"sqlite:///lightgbm_2_4_101_1000_common_truncated_20_study.db"
).best_value
feature_importances_df_common_truncated["weighted_importance"] = (best_xgboost_score * feature_importances_df_common_truncated["importance_xgboost"] + best_lightgbm_score * feature_importances_df_common_truncated["importance_lightgbm"]) / (best_xgboost_score + best_lightgbm_score)
feature_importances_df_common_truncated = feature_importances_df_common_truncated.sort_values("weighted_importance", ascending=False, ignore_index=True)
feature_importances_df_common_truncated

#### Fourth iteration: Adding popular feature on top of truncated X

In [ ]:
best_features = ['X757', 'X758', 'X759', 'X508', 'X614', 'X752', 'X331', 'X445', 'X465', 'X385', 
                 'X466', 'X95', 'X23', 'X219', 'X31', 'X373', 'X379', 'X284', 'X750', 'X652', 
                 'X279', 'X89', 'X169', 'X753', 'X226', 'X28', 'X444', 'X272', 'X271', 'X218'] + \
                ["volume", "bid_qty", "ask_qty", "buy_qty", "sell_qty"] 
# Since the features are already normalized, we cannot use the newly created features like order_flow_imbalance,
# since they lose their meanings already, but we can still use the old popular features
best_features = list(set(best_features))
train_added_df = pd.concat([train_df, popular_features_train], axis=1)
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv(train_added_df, best_features)

XGBoost

In [ ]:
best_xgboost_params_common_truncated_popular_feature = optimize_xgboost(
    f"xgboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_common_truncated_{len(best_features) - 5}_popular_feature_study",
    f"xgboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_common_truncated_{len(best_features) - 5}_popular_feature_study"
) 

LightGBM

In [ ]:
best_lightgbm_params_common_truncated_popular_feature = optimize_lightgbm(
    f"lightgbm_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_common_truncated_20_popular_feature_study",
    f"lightgbm_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_common_truncated_20_popular_feature_study"
) 

#### Fifth Iteration Instead of using GBDT, can we use MLP on these features

Convert from normal CV to torch type CV

In [ ]:
# Create the CV data, seems to be better with only anonymized features
best_features = ['X862', 'X598', 'X863', 'X856', 'X612', 'X466', 'X533', 'X861', 'X445', 'X531',
                 'X385', 'X23', 'X465', 'X284', 'X331', 'X95', 'X169', 'X285', 'X137', 'X31']
                # ["volume", "bid_qty", "ask_qty", "buy_qty", "sell_qty"] + \
                # [col for col in train_df.columns.tolist() if "X" not in col and col not in ["timestamp", "label"]]
best_features = list(set(best_features))
train_added_df = pd.concat([train_df, popular_features_train], axis=1)
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv(train_added_df, best_features)

In [ ]:
# Extra code to "reduce" from float64 to float32
def float64_to_float32(data):
    if isinstance(data, pd.DataFrame):
        for col in data.columns:
            data[col] = data[col].astype("float32")
    elif isinstance(data, pd.Series):
        data = data.astype("float32")
    return data

for i in range(default_cv):
    X_train_arr[i] = float64_to_float32(X_train_arr[i])
    X_test_arr[i] = float64_to_float32(X_test_arr[i])
    Y_train_arr[i] = float64_to_float32(Y_train_arr[i])
    Y_test_arr[i] = float64_to_float32(Y_test_arr[i])

In [ ]:
def normal_cv_to_torch_cv(X_train_arr, X_test_arr, Y_train_arr, Y_test_arr, cv=default_cv):
    train_arr = []
    test_arr = []
    for i in range(cv):
        # First shuffle the data
        X_train, Y_train = X_train_arr[i], Y_train_arr[i]
        X_train["label"] = Y_train
        # Instead of shuffle the training data when create the dataloader, try to shuffle beforehand
        # X_train = X_train.sample(frac = 1, random_state = default_random_state)
        # not shuffle, keep it by date
        Y_train = X_train["label"]
        X_train = X_train.drop("label", axis = 1)

        # Then normalize
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train.values)

        # Create train dataset
        X_train, Y_train = torch.from_numpy(X_train), torch.from_numpy(Y_train.values)
        train_dataset = TensorDataset(X_train, Y_train)
        train_arr.append(train_dataset)

        # Normalize X_test
        X_test = scaler.transform(X_test_arr[i].values)

        # Create test dataset
        X_test, Y_test = torch.from_numpy(X_test), torch.from_numpy(Y_test_arr[i].values)
        test_dataset = TensorDataset(X_test, Y_test)
        test_arr.append(test_dataset)
        
    return train_arr, test_arr

In [ ]:
train_arr, test_arr = normal_cv_to_torch_cv(X_train_arr, X_test_arr, Y_train_arr, Y_test_arr)

Define the model

In [ ]:
# Define the model
class MLP(nn.Module):
    def __init__(self, num_features, hidden_layers_size, dropout):
        super(MLP, self).__init__()

        # Initialize layers & batchnorm
        last_layer = num_features
        self.layers = nn.ModuleList()
        for current_layer in hidden_layers_size:
            self.layers.append(nn.Linear(last_layer, current_layer))
            last_layer = current_layer
        self.layers.append(nn.Linear(last_layer, 1))

        # Initialize activation
        self.activation = nn.ReLU()

        # Initialze dropout
        self.dropout = nn.Dropout(p = dropout)

    def forward(self, x):
        for inx, layer in enumerate(self.layers):
            if inx == len(self.layers) - 1:
                x = layer(x)
            else:
                x = layer(x)
                x = self.activation(x)
                x = self.dropout(x)
        return x

    def reset(self):
        for layer in self.layers:
            layer.reset_parameters()

Train model with CV and evaluate

In [ ]:
# Separate function for train & eval step
def train_mlp(model, criterion, optimizer, train_dataloader, num_epochs):
    model.train()
    for _ in tqdm(range(num_epochs)):
        for (inputs, targets) in train_dataloader:
            # Load to device
            inputs, targets= inputs.to(device), targets.to(device)
            # Forward step
            outputs = model(inputs)
            # get error
            error = criterion(outputs, targets)
            # Zero out the past gradient
            optimizer.zero_grad()
            # Backprop
            error.backward()
            # Gradient Descent
            optimizer.step()

def eval_mlp(model, test_dataloader):
    outputs_all = np.zeros(0)
    targets_all = np.zeros(0)
    model.eval()
    with torch.no_grad():
        for _, (inputs, targets) in enumerate(test_dataloader):
            # Load to device
            inputs = inputs.to(device)
            # Forward step
            outputs = model(inputs).detach().cpu().numpy().flatten()
            # Load to overall Y_test, Y_pred to calculate pearson score later
            outputs_all = np.concatenate([outputs_all, outputs])
            targets_all = np.concatenate([targets_all, targets])
    return pearson_score(targets_all, outputs_all)

In [ ]:
def train_eval_cv_torch(model, lr, cv, train_arr, test_arr, batch_size, num_epochs):
    cv_pearson = 0
    for i in range(cv):
        # Get the dataloader
        train_dataset = train_arr[i]
        train_dataloader = DataLoader(train_dataset, batch_size = batch_size, num_workers=0)
        test_dataset = test_arr[i]
        test_dataloader = DataLoader(test_dataset, batch_size = batch_size, num_workers=0)

        # Reinitialize the model
        model.reset()
        model.to(device)

        # Initialize the loss function
        criterion = nn.MSELoss()

        # Reinitialize the optimizer
        optimizer = optim.Adam(model.parameters(), lr = lr)

        # Train the model
        train_mlp(model, criterion, optimizer, train_dataloader, num_epochs)

        # Test the model
        pearson = eval_mlp(model, test_dataloader)
        print(pearson)
        cv_pearson += pearson
    return cv_pearson / cv

In [ ]:
# Training process of the default config
hidden_layers_size = [16, 8, 4]
lr = 0.001
batch_size = 60
num_epochs = 10

mlpr = MLP(len(best_features), hidden_layers_size=hidden_layers_size, dropout = 0.3)

train_eval_cv_torch(mlpr, lr, default_cv, train_arr, test_arr, batch_size, num_epochs)

#### Sixth Iteration: Change this into a classification problem

In [ ]:
original_features = [f for f in train_df.columns if "X" in f]

X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv_classification(train_df, original_features)

In [ ]:
best_params_xgboost_classification = optimize_xgboost(
    f"xgboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_classification_study",
    f"xgboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_classification_study",
    objective_xgboost_classification
)

In [ ]:
best_params_lightgbm_classification = optimize_lightgbm(
    f"lightgbm_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_classification_study",
    f"lightgbm_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_classification_study",
    objective_lightgbm_classification
)

#### Seventh Iteration: Search for the best way to train

In [ ]:
def search_training_scheme(model, train_df, cv = default_cv, features = None):
    folds_trial = [
        # level 1
        [[0, 1, 2, 3]], 
        [[0, 1]], [[1, 2]], [[2, 3]],
        [[0]], [[1]], [[2]], [[3]],
        [[0, 1], [1, 2], [2, 3]],
        [[0, 1], [2, 3]],
        [[0], [1], [2], [3]],
        # level 2
        [[0, 1, 2, 3], [0, 1]],
        [[0, 1, 2, 3], [1, 2]],
        [[0, 1, 2, 3], [2, 3]],
        [[0, 1, 2, 3], [0, 1], [2, 3]],
        [[0, 1, 2, 3], [0, 1], [1, 2], [2, 3]],
        [[0, 1, 2, 3], [0], [1], [2], [3]],
        [[0, 1], [2, 3], [0], [1], [2], [3]],
        [[0, 1], [1, 2], [2, 3], [0], [1], [2], [3]],
        # level 3
        [[0, 1, 2, 3], [0, 1], [0]],
        [[0, 1, 2, 3], [2, 3], [3]],
        [[0, 1, 2, 3], [0, 1], [2, 3], [0], [1], [2], [3]],
        [[0, 1, 2, 3], [0, 1], [1, 2], [2, 3], [0], [1], [2], [3]],
    ]

    if features is not None:
        train_df = train_df[features + ["timestamp", "label"]]

    for folds in folds_trial:
        print(f"Current folds list is {folds}")
        model_lst = [deepcopy(model)] * len(folds)
        cv_pearson = []
        for i in range(cv):
            train_month = list(range(3 + i, 7 + i))
            test_month = list(map(lambda x: x % 12 if x > 12 else x, list(range(8 + i, 12 + i))))
            test = train_df[train_df["timestamp"].dt.month.isin(test_month)].reset_index().drop("index", axis = 1)
            X_test, Y_test = test.drop(["timestamp", "label"], axis = 1), test["label"]
            Y_pred = np.zeros(Y_test.shape[0])
            for j in range(len(folds)):
                fold = folds[j]
                model = model_lst[j]
                train_month_curr = [train_month[f] for f in fold]
                train_curr = train_df[train_df["timestamp"].dt.month.isin(train_month_curr)].reset_index().drop("index", axis = 1)
                X_train, Y_train = train_curr.drop(["timestamp", "label"], axis = 1), train_curr["label"]
                model.fit(X_train, Y_train)
                Y_pred += model.predict(X_test)
            Y_pred /= len(folds)
            cv_pearson.append(pearson_score(Y_test, Y_pred))
            print(f"Finish fold {i} with score: {pearson_score(Y_test, Y_pred)}")
        print(f"Finish trial with mean score: {np.mean(np.array(cv_pearson))}")
        print(f"Finish trial with std score: {np.std(np.array(cv_pearson))}")
        print()

In [ ]:
train_added_df = pd.concat([train_df, popular_features_train], axis = 1)

params = {
    "n_estimators": default_n_trees,
    "verbosity": 0,
    "enable_categorical": True,
    "random_state": default_random_state
}
best_params_xgboost_popular_feature = get_best_params_from_file(f"xgboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_popular_feature_study")
for p in best_params_xgboost_popular_feature:
    params[p] = best_params_xgboost_popular_feature[p]

xgbr = XGBRegressor(**params)
search_training_scheme(xgbr, train_added_df)
# Notable
# [[0, 1, 2, 3]]
# [[0, 1, 2, 3], [1, 2]]
# [[0, 1, 2, 3], [0, 1], [2, 3]]
# [[0, 1, 2, 3], [0, 1], [1, 2], [2, 3]]
# [[0, 1, 2, 3], [0], [1], [2], [3]] 
# [[0, 1, 2, 3], [0, 1], [2, 3], [0], [1], [2], [3]]
# [[0, 1, 2, 3], [0, 1], [1, 2], [2, 3], [0], [1], [2], [3]]

In [ ]:
train_added_df = pd.concat([train_df, popular_features_train], axis = 1)

params = {
    "n_estimators": default_n_trees,
    "verbosity": -1,
    "random_state": default_random_state
}
best_params_lightgbm_popular_feature = get_best_params_from_file(f"lightgbm_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_popular_feature_study")
for p in best_params_lightgbm_popular_feature:
    params[p] = best_params_lightgbm_popular_feature[p]

lgbr = LGBMRegressor(**params)
search_training_scheme(lgbr, train_added_df)
# [[0, 1, 2, 3]]
# [[0, 1, 2, 3], [0, 1]]
# [[0, 1, 2, 3], [0, 1], [2, 3]]
# [[0, 1, 2, 3], [0], [1], [2], [3]]
# [[0, 1, 2, 3], [0, 1], [0]]
# [[0, 1, 2, 3], [0, 1], [2, 3], [0], [1], [2], [3]]

#### Eighth Iteration: rewrite the code for MLP training using MLX

Create the data for training + custom batch iteration

In [17]:
# Create the CV data, seems to be better with only anonymized features
# best_features = ['X862', 'X598', 'X863', 'X856', 'X612', 'X466', 'X533', 'X861', 'X445', 'X531', 
#                  'X385', 'X23', 'X284', 'X465', 'X331', 'X95', 'X285', 'X31', 'X169', 'X137', 
#                 'X379', 'X186', 'X852', 'X302', 'X868', 'X89', 'X219', 'X855', 'X540', 'X301'] 
                #  'X198', 'X373', 'X524', 'X291', 'X444', 'X279', 'X300', 'X181', 'X367', 'X538', 
                #  'X288', 'X226', 'X857', 'X860', 'X205', 'X298', 'X272', 'X472', 'X28', 'X754']
                # ["volume", "bid_qty", "ask_qty", "buy_qty", "sell_qty"] + \
                # [col for col in train_df.columns.tolist() if "X" not in col and col not in ["timestamp", "label"]]
# best_features = list(set(best_features))
# best_features = [col for col in train_df.columns if "X" in col]
# train_added_df = pd.concat([train_df, popular_features_train], axis=1)
# X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv(train_added_df, best_features)

In [18]:
# Extra code to "reduce" from float64 to float32
def float64_to_float32(data):
    if isinstance(data, pd.DataFrame):
        for col in data.columns:
            data[col] = data[col].astype("float32")
    elif isinstance(data, pd.Series):
        data = data.astype("float32")
    return data

# for i in range(default_cv):
#     X_train_arr[i] = float64_to_float32(X_train_arr[i])
#     X_test_arr[i] = float64_to_float32(X_test_arr[i])
#     Y_train_arr[i] = float64_to_float32(Y_train_arr[i])
#     Y_test_arr[i] = float64_to_float32(Y_test_arr[i])

In [19]:
def normal_cv_to_mlx_cv(X_train_arr = None, X_test_arr = None, Y_train_arr = None, Y_test_arr = None, cv=default_cv):
    for i in range(cv):
        # Normalize first
        if X_train_arr is not None and X_test_arr is not None:
            scaler = StandardScaler()
            X_train_arr[i] = scaler.fit_transform(X_train_arr[i].values)
            X_test_arr[i] = scaler.transform(X_test_arr[i].values)

        # Convert to mlx format
        if X_train_arr is not None: X_train_arr[i] = mx.array(X_train_arr[i])
        if X_test_arr is not None: X_test_arr[i] = mx.array(X_test_arr[i])
        if Y_train_arr is not None: Y_train_arr[i] = mx.array(Y_train_arr[i].values)
        if Y_test_arr is not None: Y_test_arr[i] = mx.array(Y_test_arr[i].values)
        
    return X_train_arr, X_test_arr, Y_train_arr, Y_test_arr

# X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = normal_cv_to_mlx_cv(X_train_arr, X_test_arr, Y_train_arr, Y_test_arr)

Define the model

In [20]:
# Define the model
# We do not use the reset method this time so you have to create the model at each fold
class MLPMLX(nnmx.Module):
    def __init__(self, num_features, hidden_layers_size, dropout):
        super().__init__()

        # Initialize layers & batchnorm
        last_layer = num_features
        self.layers = []
        for current_layer in hidden_layers_size:
            self.layers.append(nnmx.Linear(last_layer, current_layer))
            last_layer = current_layer
        self.layers.append(nnmx.Linear(last_layer, 1))

        # Initialize activation
        self.activation = nnmx.ReLU()

        # Initialize dropout
        self.dropout = nnmx.Dropout(p = dropout)

    def __call__(self, x):
        for inx, layer in enumerate(self.layers):
            x = layer(x)
            if inx != len(self.layers) - 1:
                x = self.activation(x)
                x = self.dropout(x)
        return x

Train model with CV and evaluate

In [21]:
# Custom function for batch iteration
def batch_iterate(batch_size, X, Y, shuffle = True):
    for i in range(0, Y.size, batch_size):
        X_curr = X[i: min(i + batch_size, Y.size), :]
        Y_curr = Y[i: min(i + batch_size, Y.size)]
        if shuffle:
            inx_lst = mx.random.permutation(batch_size)
            X_curr = X_curr[inx_lst, :]
            Y_curr = Y_curr[inx_lst]
        yield X_curr, Y_curr

In [22]:
# Separate function for train & eval step
def train_mlp_mlx(model, loss_and_grad_fn, optimizer, X_train, Y_train, batch_size, num_epochs):
    model.train()
    for _ in tqdm(range(num_epochs)):
        for (inputs, targets) in batch_iterate(batch_size, X_train, Y_train):
            _, grads = loss_and_grad_fn(model, inputs, targets)
            # Update the optimizer state and model parameters in a single call
            optimizer.update(model, grads)
            # Force a graph evaluation
            mx.eval(model.parameters(), optimizer.state)

def eval_mlp_mlx(model, X_test, Y_test, batch_size):
    outputs_all = np.zeros(0)
    targets_all = np.zeros(0)
    model.eval()
    for (inputs, targets) in batch_iterate(batch_size, X_test, Y_test, shuffle=False):
        outputs = model(inputs).reshape(-1)
        # convert back to numpy
        outputs, targets = np.array(outputs), np.array(targets)
        # Load to overall Y_test, Y_pred to calculate pearson score later
        outputs_all = np.concatenate([outputs_all, outputs])
        targets_all = np.concatenate([targets_all, targets])
    return pearson_score(targets_all, outputs_all)

In [23]:
def train_eval_cv_mlx(num_features, hidden_layers_size, dropout, lr, cv, X_train_arr, X_test_arr, Y_train_arr, Y_test_arr, batch_size, num_epochs):
    cv_pearson = 0
    for _, (X_train, X_test, Y_train, Y_test) in enumerate(zip(X_train_arr, X_test_arr, Y_train_arr, Y_test_arr)):
        # initialize the model
        mx.random.seed(default_random_state)
        model = MLPMLX(num_features, hidden_layers_size, dropout)

        # Initialize the loss function
        def loss_fn(model, X, Y):
            Y_pred = model(X).reshape(-1)
            return mx.mean(nnmx.losses.mse_loss(Y_pred, Y))
            # Y_centered = Y - mx.mean(Y)
            # Y_pred_centered = Y_pred - mx.mean(Y_pred)
            # return mx.sum(Y_centered * Y_pred_centered) / mx.sqrt(mx.sum(Y_centered ** 2) * mx.sum(Y_pred_centered ** 2))
        loss_and_grad_fn = nnmx.value_and_grad(model, loss_fn)

        # Reinitialize the optimizer
        optimizer = optimmx.Adam(learning_rate = lr)

        # Train the model
        train_mlp_mlx(model, loss_and_grad_fn, optimizer, X_train, Y_train, batch_size, num_epochs)

        # Test the model
        pearson = eval_mlp_mlx(model, X_test, Y_test, batch_size)
        print(pearson)
        if pearson == -1:
            return pearson
        cv_pearson += pearson
    return cv_pearson / cv

Conduct training and evaluating process of the model

In [24]:
# # Training process of the default config
# num_features = len(best_features)
# hidden_layers_size = [8, 8, 8]
# dropout = 0.2
# lr = 0.001
# batch_size = 180
# num_epochs = 10

# train_eval_cv_mlx(num_features, hidden_layers_size, dropout, lr, default_cv, X_train_arr, X_test_arr, Y_train_arr, Y_test_arr, batch_size, num_epochs)

Conduct Bayesian Optimization on this

In [25]:
default_num_layers = 2

In [26]:
def objective_mlp_mlx(trial):
    # First initialize the parameters
    num_features = len(best_features)
    num_layers = default_num_layers
    log_2_hidden_layers_size = []
    for i in range(num_layers):
        if len(log_2_hidden_layers_size) == 0:
            log_2_hidden_layers_size.append(trial.suggest_int(f"log2_hidden_layer_{i}", 2, 7))
        else:
            log_2_hidden_layers_size.append(trial.suggest_int(f"log2_hidden_layer_{i}", 2, log_2_hidden_layers_size[-1]))
    hidden_layers_size = [2**l for l in log_2_hidden_layers_size]
    dropout = trial.suggest_categorical("dropout", [0.2, 0.3, 0.4, 0.5, 0.6, 0.7])
    lr = trial.suggest_float("lr", 0.0001, 0.01, log=True)
    batch_size = trial.suggest_categorical("batch_size", [30, 60, 120, 180, 360, 720])
    num_epochs = trial.suggest_categorical("num_epochs", [5 * i for i in range(1, 7)])
    
    # Conduct training based on those parameters
    return train_eval_cv_mlx(num_features, hidden_layers_size, dropout, lr, default_cv, X_train_arr, X_test_arr, Y_train_arr, Y_test_arr, batch_size, num_epochs)

In [30]:
def optimize_mlp_mlx(study_name, storage_name, objective_function=objective_mlp_mlx, n_trials = 100, n_jobs = 1):
    print("Conduct hyperparam opt for MLP")
    study = optuna.create_study(
        study_name = study_name,
        direction ='maximize',
        storage = f"sqlite:///{storage_name}.db",
        sampler = TPESampler(seed = 101, n_startup_trials=10),
        load_if_exists=True
    )
    study.optimize(objective_function, n_trials=n_trials, n_jobs=n_jobs)
    print('Best hyperparameters:', study.best_params)
    print('Best Pearson score:', study.best_value)
    return study.best_params

In [31]:
# Create the CV data, seems to be better with only anonymized features
best_features = ['X757', 'X758', 'X759', 'X508', 'X614', 'X752', 'X331', 'X445', 'X465', 'X385', 
                 'X466', 'X95', 'X23', 'X219', 'X31', 'X373', 'X379', 'X284', 'X750', 'X652', 
                 'X279', 'X89', 'X169', 'X753', 'X226', 'X28', 'X444', 'X272', 'X271', 'X218']
                # ["volume", "bid_qty", "ask_qty", "buy_qty", "sell_qty"] + \
                # [col for col in train_df.columns.tolist() if "X" not in col and col not in ["timestamp", "label"]]
best_features = list(set(best_features))
# best_features = [col for col in train_df.columns if "X" in col]
# train_added_df = pd.concat([train_df, popular_features_train], axis=1)
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv(train_df, best_features)

# Convert to float32
for i in range(default_cv):
    X_train_arr[i] = float64_to_float32(X_train_arr[i])
    X_test_arr[i] = float64_to_float32(X_test_arr[i])
    Y_train_arr[i] = float64_to_float32(Y_train_arr[i])
    Y_test_arr[i] = float64_to_float32(Y_test_arr[i])

# Convert to MLX
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = normal_cv_to_mlx_cv(X_train_arr, X_test_arr, Y_train_arr, Y_test_arr)

[3, 4, 5, 6] [8, 9, 10, 11]
[4, 5, 6, 7] [9, 10, 11, 12]
[5, 6, 7, 8] [10, 11, 12, 1]
[6, 7, 8, 9] [11, 12, 1, 2]


In [33]:
optimize_mlp_mlx(
    f"mlp_mlx_{feature_version}_{default_cv}_{default_random_state}_{default_num_layers}_common_truncated_{len(best_features)}_study",
    f"mlp_mlx_{feature_version}_{default_cv}_{default_random_state}_{default_num_layers}_common_truncated_{len(best_features)}_study"
)

[I 2025-07-16 15:41:17,316] Using an existing study with name 'mlp_mlx_2_4_101_2_common_truncated_30_study' instead of creating a new one.


Conduct hyperparam opt for MLP


100%|██████████| 15/15 [00:19<00:00,  1.29s/it]


0.10823771394955192


100%|██████████| 15/15 [00:18<00:00,  1.22s/it]


...

100%|██████████| 15/15 [00:11<00:00,  1.27it/s]


0.12172668056126239


100%|██████████| 15/15 [00:11<00:00,  1.26it/s]
[I 2025-07-16 17:12:38,057] Trial 300 finished with value: 0.12423102372845835 and parameters: {'log2_hidden_layer_0': 5, 'log2_hidden_layer_1': 3, 'dropout': 0.6, 'lr': 0.0001290187070108035, 'batch_size': 180, 'num_epochs': 15}. Best is trial 63 with value: 0.12508327601680538.


0.12900687082186155
Best hyperparameters: {'log2_hidden_layer_0': 5, 'log2_hidden_layer_1': 3, 'dropout': 0.2, 'lr': 0.0001159469267719501, 'batch_size': 360, 'num_epochs': 15}
Best Pearson score: 0.12508327601680538


{'log2_hidden_layer_0': 5,
 'log2_hidden_layer_1': 3,
 'dropout': 0.2,
 'lr': 0.0001159469267719501,
 'batch_size': 360,
 'num_epochs': 15}

#### Nineth Iteration: AE + MLP instead of GBDT feature selection + MLP (train together)

Define the model

In [34]:
# Define gaussian noise for autoencoder
class GaussianNoise(nnmx.Module):
    def __init__(self, mean: float = 0.0, stddev: float = 0.01):
        super().__init__()
        self.mean = mean
        self.stddev = stddev

    def __call__(self, x, training = True):
        if training:
            x += mx.random.normal(loc=self.mean, scale=self.stddev, shape=x.shape)
        return x

In [35]:
# Define the model
# We do not use the reset method this time so you have to create the model at each fold
class AEMLX(nnmx.Module):
    def __init__(self, num_features, hidden_layers_size, latent_size, dropout):
        super().__init__()

        # Initialize layers for encoder
        last_layer = num_features
        self.encoder_layers = []
        for current_layer in hidden_layers_size:
            self.encoder_layers.append(nnmx.Linear(last_layer, current_layer))
            last_layer = current_layer
        self.encoder_layers.append(nnmx.Linear(last_layer, latent_size))

        # Initialize layers for decoder
        last_layer = latent_size
        self.decoder_layers = []
        for current_layer in hidden_layers_size[::-1]:
            self.decoder_layers.append(nnmx.Linear(last_layer, current_layer))
            last_layer = current_layer
        self.decoder_layers.append(nnmx.Linear(last_layer, num_features))

        # Initialize activation
        self.activation = nnmx.ReLU()

        # Initialze gaussian noise to apply upon training
        # self.gaussian_noise = GaussianNoise()

        # Initialize dropout
        self.dropout = nnmx.Dropout(p = dropout)

    def __call__(self, x, training = True):
        # if training:
        #     x = self.gaussian_noise(x)
            
        for inx, layer in enumerate(self.encoder_layers):
            x = layer(x)
            x = self.activation(x)
            x = self.dropout(x)
        for inx, layer in enumerate(self.decoder_layers):
            if inx == len(self.decoder_layers) - 1:
                x = layer(x)
            else:
                x = layer(x)
                x = self.activation(x)
                x = self.dropout(x)
        return x

    def get_latent(self, x):
        for inx, layer in enumerate(self.encoder_layers):
            x = layer(x)
            x = self.activation(x)
        return x

Train model with CV and evaluate

In [36]:
# Separate function for train & eval step
def train_aemlp_mlx(ae_model, ae_loss_and_grad_fn, ae_optimizer, ae_num_epochs,
                     mlp_model, mlp_loss_and_grad_fn, mlp_optimizer, mlp_num_epochs,
                     X_train, Y_train, batch_size):
    # Train ae first
    ae_model.train()
    for _ in tqdm(range(ae_num_epochs)):
        for (inputs, targets) in batch_iterate(batch_size, X_train, Y_train):
            # get gradients for ae, output is the inputs itself
            _, ae_grads = ae_loss_and_grad_fn(ae_model, inputs, inputs)

            # Update the optimizer state and model parameters in a single call
            ae_optimizer.update(ae_model, ae_grads)

            # Force a graph evaluation
            mx.eval(ae_model.parameters(), ae_optimizer.state)

    # Train mlp later
    mlp_model.train()
    for _ in tqdm(range(mlp_num_epochs)):
        for (inputs, targets) in batch_iterate(batch_size, X_train, Y_train):
            # get the latent representation for X_train
            latent_inputs = ae_model.get_latent(inputs)
            used_inputs = mx.concatenate([inputs, latent_inputs], axis=1)
            # get gradients for mlp
            _, mlp_grads = mlp_loss_and_grad_fn(mlp_model, used_inputs, targets)

            # Update the optimizer state and model parameters in a single call
            mlp_optimizer.update(mlp_model, mlp_grads)

            # Force a graph evaluation
            mx.eval(mlp_model.parameters(), mlp_optimizer.state)

    # # Train ae and mlp together
    # ae_model.train()
    # mlp_model.train()
    # for _ in tqdm(range(ae_num_epochs)):
    #     for (inputs, targets) in batch_iterate(batch_size, X_train, Y_train):
    #         # get gradients for ae, output is the inputs itself
    #         _, ae_grads = ae_loss_and_grad_fn(ae_model, inputs, inputs)

    #         # Update the optimizer state and model parameters in a single call
    #         ae_optimizer.update(ae_model, ae_grads)

    #         # Force a graph evaluation
    #         mx.eval(ae_model.parameters(), ae_optimizer.state)

    #         # get gradients for mlp
    #         latent_inputs = ae_model.get_latent(inputs)
    #         used_inputs = mx.concatenate([inputs, latent_inputs], axis=1)
    #         _, mlp_grads = mlp_loss_and_grad_fn(mlp_model, used_inputs, targets)

    #         # Update the optimizer state and model parameters in a single call
    #         mlp_optimizer.update(mlp_model, mlp_grads)

    #         # Force a graph evaluation
    #         mx.eval(mlp_model.parameters(), mlp_optimizer.state)

def eval_aemlp_mlx(ae_model, mlp_model, X_test, Y_test, batch_size):
    outputs_all = np.zeros(0)
    targets_all = np.zeros(0)
    ae_model.eval()
    mlp_model.eval()
    for (inputs, targets) in batch_iterate(batch_size, X_test, Y_test, shuffle=False):
        latent_inputs = ae_model.get_latent(inputs)
        used_inputs = mx.concatenate([inputs, latent_inputs], axis=1)
        outputs = mlp_model(used_inputs).reshape(-1)
        # convert back to numpy
        outputs, targets = np.array(outputs), np.array(targets)
        # Load to overall Y_test, Y_pred to calculate pearson score later
        outputs_all = np.concatenate([outputs_all, outputs])
        targets_all = np.concatenate([targets_all, targets])
    return pearson_score(targets_all, outputs_all)

In [37]:
def train_eval_cv_mlx_aemlp(num_features, ae_hidden_layers_size, ae_latent_size, ae_dropout, ae_lr, ae_num_epochs,
                            mlp_hidden_layers_size, mlp_dropout, mlp_lr, mlp_num_epochs,
                            cv, X_train_arr, X_test_arr, Y_train_arr, Y_test_arr, batch_size):
    cv_pearson = 0
    for _, (X_train, X_test, Y_train, Y_test) in enumerate(zip(X_train_arr, X_test_arr, Y_train_arr, Y_test_arr)):
        # initialize the model
        mx.random.seed(default_random_state)
        ae_model = AEMLX(num_features, ae_hidden_layers_size, ae_latent_size, ae_dropout)

        mx.random.seed(default_random_state)
        mlp_model = MLPMLX(ae_latent_size + num_features, mlp_hidden_layers_size, mlp_dropout)

        # Initialize the loss function (both use same loss function)
        def loss_fn(model, X, Y):
            Y_pred = model(X).reshape(-1)
            Y = Y.reshape(-1)
            return mx.mean(nnmx.losses.mse_loss(Y_pred, Y))
        ae_loss_and_grad_fn = nnmx.value_and_grad(ae_model, loss_fn)
        mlp_loss_and_grad_fn = nnmx.value_and_grad(mlp_model, loss_fn)

        # Reinitialize the optimizer
        ae_optimizer = optimmx.Adam(learning_rate = ae_lr)
        mlp_optimizer = optimmx.Adam(learning_rate = mlp_lr)

        # Train the model
        train_aemlp_mlx(ae_model, ae_loss_and_grad_fn, ae_optimizer, ae_num_epochs,
                        mlp_model, mlp_loss_and_grad_fn, mlp_optimizer, mlp_num_epochs,
                        X_train, Y_train, batch_size)

        # Test the model
        pearson = eval_aemlp_mlx(ae_model, mlp_model, X_test, Y_test, batch_size)
        print(pearson)
        if pearson == -1:
            return pearson
        cv_pearson += pearson
    return cv_pearson / cv

Conduct training and evaluating process of the model

In [38]:
# # Create the CV data, seems to be better with only anonymized features
# best_features = [col for col in train_df.columns if "X" in col] + \
#                 ["volume", "bid_qty", "ask_qty", "buy_qty", "sell_qty"] + \
#                 [col for col in train_df.columns.tolist() if "X" not in col and col not in ["timestamp", "label"]]
# train_added_df = pd.concat([train_df, popular_features_train], axis=1)
# X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv(train_added_df, best_features)
# for i in range(default_cv):
#     X_train_arr[i] = float64_to_float32(X_train_arr[i])
#     X_test_arr[i] = float64_to_float32(X_test_arr[i])
#     Y_train_arr[i] = float64_to_float32(Y_train_arr[i])
#     Y_test_arr[i] = float64_to_float32(Y_test_arr[i])
# X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = normal_cv_to_mlx_cv(X_train_arr, X_test_arr, Y_train_arr, Y_test_arr)

In [39]:
# # # Training process of the default config
# num_features = len(best_features)
# ae_hidden_layers_size = [64]
# ae_latent_size = 16
# mlp_hidden_layers_size = [4, 2]
# lr = 0.0005
# dropout = 0.5
# batch_size = 180
# num_epochs = 30

# train_eval_cv_mlx_aemlp(num_features, ae_hidden_layers_size, ae_latent_size, 
#                         mlp_hidden_layers_size, dropout, 
#                         lr, default_cv,
#                         X_train_arr, X_test_arr, Y_train_arr, Y_test_arr,
#                         batch_size, num_epochs)

Optimize with bayesian optimization

In [40]:
ae_default_num_layers = 2
mlp_default_num_layers = 1

In [41]:
def objective_aemlp_mlx(trial):
    # First initialize the parameters
    num_features = len(best_features)

    # initialize ae layers
    ae_num_layers = ae_default_num_layers
    ae_log_2_hidden_layers_size = []
    for i in range(ae_num_layers):
        if len(ae_log_2_hidden_layers_size) == 0:
            ae_log_2_hidden_layers_size.append(trial.suggest_int(f"ae_log2_hidden_layer_{i}", 3, int(math.ceil(math.log2(num_features)))))
        else:
            ae_log_2_hidden_layers_size.append(trial.suggest_int(f"ae_log2_hidden_layer_{i}", 3, ae_log_2_hidden_layers_size[-1]))
    ae_hidden_layers_size = [2**i for i in ae_log_2_hidden_layers_size]
    ae_latent_size = 2**trial.suggest_int("ae_log2_latent_size", 3, ae_log_2_hidden_layers_size[-1])
    ae_dropout = trial.suggest_categorical("ae_dropout", [0.2, 0.3, 0.4, 0.5, 0.6, 0.7])
    ae_lr = trial.suggest_float("ae_lr", 0.0001, 0.01, log=True)
    ae_num_epochs = trial.suggest_categorical("num_epochs", [5 * i for i in range(1, 7)])

    # initialize mlp layers
    mlp_num_layers = mlp_default_num_layers
    mlp_log_2_hidden_layers_size = []
    for i in range(mlp_num_layers):
        if len(mlp_log_2_hidden_layers_size) == 0:
            mlp_log_2_hidden_layers_size.append(trial.suggest_int(f"mlp_log2_hidden_layer_{i}", 2, int(math.ceil(math.log2(ae_latent_size + num_features)))))
        else:
            mlp_log_2_hidden_layers_size.append(trial.suggest_int(f"mlp_log2_hidden_layer_{i}", 2, mlp_log_2_hidden_layers_size[-1]))
    mlp_hidden_layers_size = [2**i for i in mlp_log_2_hidden_layers_size]
    mlp_dropout = trial.suggest_categorical("mle_dropout", [0.2, 0.3, 0.4, 0.5, 0.6, 0.7])
    mlp_lr = trial.suggest_float("mlp_lr", 0.0001, 0.01, log=True)
    # mlp_num_epochs = trial.suggest_categorical("mlp_num_epochs", [10, 20, 30, 40, 50])
    mlp_num_epochs = ae_num_epochs

    # batch size
    batch_size = trial.suggest_categorical("batch_size", [30, 60, 120, 180, 360, 720, 1440])
    
    # Conduct training based on those parameters
    cv_pearson = train_eval_cv_mlx_aemlp(num_features, ae_hidden_layers_size, ae_latent_size, ae_dropout, ae_lr, ae_num_epochs,
                                         mlp_hidden_layers_size, mlp_dropout, mlp_lr, mlp_num_epochs,
                                         default_cv, X_train_arr, X_test_arr, Y_train_arr, Y_test_arr,
                                         batch_size)
    
    return cv_pearson

In [42]:
def optimize_aemlp_mlx(study_name, storage_name, objective_function=objective_aemlp_mlx, n_trials = 100, n_jobs = 1):
    print("Conduct hyperparam opt for AE-MLP")
    study = optuna.create_study(
        study_name = study_name,
        direction ='maximize',
        storage = f"sqlite:///{storage_name}.db",
        sampler = TPESampler(seed = 101, n_startup_trials=10),
        load_if_exists=True
    )
    study.optimize(objective_function, n_trials=n_trials, n_jobs=n_jobs)
    print('Best hyperparameters:', study.best_params)
    print('Best Pearson score:', study.best_value)
    return study.best_params

In [43]:
best_features = [col for col in train_df.columns if "X" in col and "interaction" not in col]
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv(train_df, best_features)
for i in range(default_cv):
    X_train_arr[i] = float64_to_float32(X_train_arr[i])
    X_test_arr[i] = float64_to_float32(X_test_arr[i])
    Y_train_arr[i] = float64_to_float32(Y_train_arr[i])
    Y_test_arr[i] = float64_to_float32(Y_test_arr[i])
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = normal_cv_to_mlx_cv(X_train_arr, X_test_arr, Y_train_arr, Y_test_arr)

[3, 4, 5, 6] [8, 9, 10, 11]
[4, 5, 6, 7] [9, 10, 11, 12]
[5, 6, 7, 8] [10, 11, 12, 1]
[6, 7, 8, 9] [11, 12, 1, 2]


In [ ]:
optimize_aemlp_mlx(
    f"aemlp_mlx_{feature_version}_{default_cv}_{default_random_state}_{ae_default_num_layers}_{mlp_default_num_layers}_study",
    f"aemlp_mlx_{feature_version}_{default_cv}_{default_random_state}_{ae_default_num_layers}_{mlp_default_num_layers}_study"
)

[I 2025-07-16 17:12:39,809] A new study created in RDB with name: aemlp_mlx_2_4_101_2_1_study


Conduct hyperparam opt for AE-MLP


100%|██████████| 25/25 [00:44<00:00,  1.79s/it]


0.07083862792685322


100%|██████████| 25/25 [00:44<00:00,  1.80s/it]


...

100%|██████████| 20/20 [00:27<00:00,  1.38s/it]


0.006569495779550258


100%|██████████| 20/20 [10:59<00:00, 32.98s/it] 
[I 2025-07-16 18:16:22,213] Trial 7 finished with value: 0.04379681758856209 and parameters: {'ae_log2_hidden_layer_0': 3, 'ae_log2_hidden_layer_1': 3, 'ae_log2_latent_size': 3, 'ae_dropout': 0.6, 'ae_lr': 0.001651927639666053, 'num_epochs': 20, 'mlp_log2_hidden_layer_0': 4, 'mle_dropout': 0.2, 'mlp_lr': 0.008607749987514188, 'batch_size': 180}. Best is trial 6 with value: 0.0981172522534084.


0.032096525956807766


 53%|█████▎    | 16/30 [02:42<02:15,  9.67s/it]

#### Tenth Trial: Adding interaction terms to models

In [ ]:
best_features = ['X757', 'X758', 'X759', 'X508', 'X614', 'X752', 'X331', 'X445', 'X465', 'X385', 
                 'X466', 'X95', 'X23', 'X219', 'X31', 'X373', 'X379', 'X284', 'X750', 'X652', 
                 'X279', 'X89', 'X169', 'X753', 'X226', 'X28', 'X444', 'X272', 'X271', 'X218']
best_interaction_features = ['X758_X508_X219_interaction', 'X758_X508_X218_interaction', 'X758_X219_X89_interaction']

X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv(train_df, best_features + best_interaction_features)

XGBoost

In [ ]:
best_xgboost_params_common_truncated_with_interaction = optimize_xgboost(
    f"xgboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_common_truncated_{len(best_features)}_with_interaction_study",
    f"xgboost_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_common_truncated_{len(best_features)}_with_interaction_study"
) 

LightGBM

In [ ]:
best_lightgbm_params_common_truncated_with_interaction = optimize_lightgbm(
    f"lightgbm_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_common_truncated_{len(best_features)}_with_interaction_study",
    f"lightgbm_{feature_version}_{default_cv}_{default_random_state}_{default_n_trees}_common_truncated_{len(best_features)}_with_interaction_study"
) 

MLP

In [ ]:
default_num_layers = 2
# Create the CV data, seems to be better with only anonymized features
best_features = ['X757', 'X758', 'X759', 'X508', 'X614', 'X752', 'X331', 'X445', 'X465', 'X385', 
                 'X466', 'X95', 'X23', 'X219', 'X31', 'X373', 'X379', 'X284', 'X750', 'X652', 
                 'X279', 'X89', 'X169', 'X753', 'X226', 'X28', 'X444', 'X272', 'X271', 'X218']
                # ["volume", "bid_qty", "ask_qty", "buy_qty", "sell_qty"] + \
                # [col for col in train_df.columns.tolist() if "X" not in col and col not in ["timestamp", "label"]]
best_interaction_features = ['X758_X508_X219_interaction', 'X758_X508_X218_interaction', 'X758_X219_X89_interaction']
best_features = list(set(best_features + best_interaction_features))
# best_features = [col for col in train_df.columns if "X" in col]
# train_added_df = pd.concat([train_df, popular_features_train], axis=1)
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv(train_df, best_features)

# Convert to float32
for i in range(default_cv):
    X_train_arr[i] = float64_to_float32(X_train_arr[i])
    X_test_arr[i] = float64_to_float32(X_test_arr[i])
    Y_train_arr[i] = float64_to_float32(Y_train_arr[i])
    Y_test_arr[i] = float64_to_float32(Y_test_arr[i])

# Convert to MLX
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = normal_cv_to_mlx_cv(X_train_arr, X_test_arr, Y_train_arr, Y_test_arr)

In [ ]:
optimize_mlp_mlx(
    f"mlp_mlx_{feature_version}_{default_cv}_{default_random_state}_{default_num_layers}_common_truncated_30_with_interaction_study",
    f"mlp_mlx_{feature_version}_{default_cv}_{default_random_state}_{default_num_layers}_common_truncated_30_with_interaction_study"
)

#### Eleventh Trial: AE-MLP but MLP step is concat of good features + AE features

In [ ]:
# Custom function for batch iteration
def batch_iterate_with_concat(batch_size, X, Y, X_good, shuffle = True):
    for i in range(0, Y.size, batch_size):
        X_curr = X[i: min(i + batch_size, Y.size), :]
        Y_curr = Y[i: min(i + batch_size, Y.size)]
        X_good_curr = X_good[i: min(i + batch_size, Y.size), :]
        if shuffle:
            inx_lst = mx.random.permutation(batch_size)
            X_curr = X_curr[inx_lst, :]
            Y_curr = Y_curr[inx_lst]
            X_good_curr = X_good_curr[inx_lst, :]
        yield X_curr, Y_curr, X_good_curr

In [ ]:
# Separate function for train & eval step
def train_aemlp_mlx_with_concat(ae_model, ae_loss_and_grad_fn, ae_optimizer, ae_num_epochs,
                                mlp_model, mlp_loss_and_grad_fn, mlp_optimizer, mlp_num_epochs,
                                X_train, Y_train, X_train_good, batch_size):
    # Train ae first
    ae_model.train()
    for _ in tqdm(range(ae_num_epochs)):
        for (inputs, targets) in batch_iterate(batch_size, X_train, Y_train):
            # get gradients for ae, output is the inputs itself
            _, ae_grads = ae_loss_and_grad_fn(ae_model, inputs, inputs)

            # Update the optimizer state and model parameters in a single call
            ae_optimizer.update(ae_model, ae_grads)

            # Force a graph evaluation
            mx.eval(ae_model.parameters(), ae_optimizer.state)

    # Train mlp later
    mlp_model.train()
    for _ in tqdm(range(mlp_num_epochs)):
        for (inputs, targets, inputs_good) in batch_iterate_with_concat(batch_size, X_train, Y_train, X_train_good):
            # get the latent representation for X_train
            latent_inputs = ae_model.get_latent(inputs)
            used_inputs = mx.concatenate([inputs_good, latent_inputs], axis=1)
            # get gradients for mlp
            _, mlp_grads = mlp_loss_and_grad_fn(mlp_model, used_inputs, targets)

            # Update the optimizer state and model parameters in a single call
            mlp_optimizer.update(mlp_model, mlp_grads)

            # Force a graph evaluation
            mx.eval(mlp_model.parameters(), mlp_optimizer.state)

    # # Train ae and mlp together
    # ae_model.train()
    # mlp_model.train()
    # for _ in tqdm(range(ae_num_epochs)):
    #     for (inputs, targets) in batch_iterate(batch_size, X_train, Y_train):
    #         # get gradients for ae, output is the inputs itself
    #         _, ae_grads = ae_loss_and_grad_fn(ae_model, inputs, inputs)

    #         # Update the optimizer state and model parameters in a single call
    #         ae_optimizer.update(ae_model, ae_grads)

    #         # Force a graph evaluation
    #         mx.eval(ae_model.parameters(), ae_optimizer.state)

    #         # get gradients for mlp
    #         latent_inputs = ae_model.get_latent(inputs)
    #         used_inputs = mx.concatenate([inputs, latent_inputs], axis=1)
    #         _, mlp_grads = mlp_loss_and_grad_fn(mlp_model, used_inputs, targets)

    #         # Update the optimizer state and model parameters in a single call
    #         mlp_optimizer.update(mlp_model, mlp_grads)

    #         # Force a graph evaluation
    #         mx.eval(mlp_model.parameters(), mlp_optimizer.state)

def eval_aemlp_mlx_with_concat(ae_model, mlp_model, X_test, Y_test, X_test_good, batch_size):
    outputs_all = np.zeros(0)
    targets_all = np.zeros(0)
    ae_model.eval()
    mlp_model.eval()
    for (inputs, targets, inputs_good) in batch_iterate_with_concat(batch_size, X_test, Y_test, X_test_good, shuffle=False):
        latent_inputs = ae_model.get_latent(inputs)
        used_inputs = mx.concatenate([inputs_good, latent_inputs], axis=1)
        outputs = mlp_model(used_inputs).reshape(-1)
        # convert back to numpy
        outputs, targets = np.array(outputs), np.array(targets)
        # Load to overall Y_test, Y_pred to calculate pearson score later
        outputs_all = np.concatenate([outputs_all, outputs])
        targets_all = np.concatenate([targets_all, targets])
    return pearson_score(targets_all, outputs_all)

In [ ]:
def train_eval_cv_mlx_aemlp_with_concat(num_features, ae_hidden_layers_size, ae_latent_size, ae_dropout, ae_lr, ae_num_epochs,
                                        mlp_hidden_layers_size, mlp_dropout, mlp_lr, mlp_num_epochs,
                                        cv, X_train_arr, X_test_arr, Y_train_arr, Y_test_arr, 
                                        X_train_good_arr, X_test_good_arr, batch_size):
    cv_pearson = 0
    for _, (X_train, X_test, Y_train, Y_test, X_train_good, X_test_good) in enumerate(zip(X_train_arr, X_test_arr, Y_train_arr, Y_test_arr, X_train_good_arr, X_test_good_arr)):
        # initialize the model
        mx.random.seed(default_random_state)
        ae_model = AEMLX(num_features, ae_hidden_layers_size, ae_latent_size, ae_dropout)

        mx.random.seed(default_random_state)
        mlp_model = MLPMLX(ae_latent_size + X_train_good.shape[1], mlp_hidden_layers_size, mlp_dropout)

        # Initialize the loss function (both use same loss function)
        def loss_fn(model, X, Y):
            Y_pred = model(X).reshape(-1)
            Y = Y.reshape(-1)
            return mx.mean(nnmx.losses.mse_loss(Y_pred, Y))
        ae_loss_and_grad_fn = nnmx.value_and_grad(ae_model, loss_fn)
        mlp_loss_and_grad_fn = nnmx.value_and_grad(mlp_model, loss_fn)

        # Reinitialize the optimizer
        ae_optimizer = optimmx.Adam(learning_rate = ae_lr)
        mlp_optimizer = optimmx.Adam(learning_rate = mlp_lr)

        # Train the model
        train_aemlp_mlx_with_concat(ae_model, ae_loss_and_grad_fn, ae_optimizer, ae_num_epochs,
                                    mlp_model, mlp_loss_and_grad_fn, mlp_optimizer, mlp_num_epochs,
                                    X_train, Y_train, X_train_good, batch_size)

        # Test the model
        pearson = eval_aemlp_mlx_with_concat(ae_model, mlp_model, X_test, Y_test, X_test_good, batch_size)
        print(pearson)
        if pearson == -1:
            return pearson
        cv_pearson += pearson
    return cv_pearson / cv

In [ ]:
def objective_aemlp_mlx_with_concat(trial):
    # First initialize the parameters
    num_features = len(best_features)

    # initialize ae layers
    ae_num_layers = ae_default_num_layers
    ae_log_2_hidden_layers_size = []
    for i in range(ae_num_layers):
        if len(ae_log_2_hidden_layers_size) == 0:
            ae_log_2_hidden_layers_size.append(trial.suggest_int(f"ae_log2_hidden_layer_{i}", 3, int(math.ceil(math.log2(num_features)))))
        else:
            ae_log_2_hidden_layers_size.append(trial.suggest_int(f"ae_log2_hidden_layer_{i}", 3, ae_log_2_hidden_layers_size[-1]))
    ae_hidden_layers_size = [2**i for i in ae_log_2_hidden_layers_size]
    ae_latent_size = 2**trial.suggest_int("ae_log2_latent_size", 3, ae_log_2_hidden_layers_size[-1])
    ae_dropout = trial.suggest_categorical("ae_dropout", [0.2, 0.3, 0.4, 0.5, 0.6, 0.7])
    ae_lr = trial.suggest_float("ae_lr", 0.0001, 0.01, log=True)
    ae_num_epochs = trial.suggest_categorical("num_epochs", [10, 20, 30, 40, 50])

    # initialize mlp layers
    mlp_num_layers = mlp_default_num_layers
    mlp_log_2_hidden_layers_size = []
    for i in range(mlp_num_layers):
        if len(mlp_log_2_hidden_layers_size) == 0:
            mlp_log_2_hidden_layers_size.append(trial.suggest_int(f"mlp_log2_hidden_layer_{i}", 2, int(math.ceil(math.log2(ae_latent_size + num_features)))))
        else:
            mlp_log_2_hidden_layers_size.append(trial.suggest_int(f"mlp_log2_hidden_layer_{i}", 2, mlp_log_2_hidden_layers_size[-1]))
    mlp_hidden_layers_size = [2**i for i in mlp_log_2_hidden_layers_size]
    mlp_dropout = trial.suggest_categorical("mlp_dropout", [0.2, 0.3, 0.4, 0.5, 0.6, 0.7])
    mlp_lr = trial.suggest_float("mlp_lr", 0.0001, 0.01, log=True)
    # mlp_num_epochs = trial.suggest_categorical("mlp_num_epochs", [10, 20, 30, 40, 50])
    mlp_num_epochs = ae_num_epochs

    # batch size
    batch_size = trial.suggest_categorical("batch_size", [30, 60, 120, 180, 360, 720, 1440])
    
    # Conduct training based on those parameters
    cv_pearson = train_eval_cv_mlx_aemlp_with_concat(num_features, ae_hidden_layers_size, ae_latent_size, ae_dropout, ae_lr, ae_num_epochs,
                                                     mlp_hidden_layers_size, mlp_dropout, mlp_lr, mlp_num_epochs,
                                                     default_cv, X_train_arr, X_test_arr, Y_train_arr, Y_test_arr,
                                                     X_train_good_arr, X_test_good_arr, batch_size)
    
    return cv_pearson

In [ ]:
best_features = [col for col in train_df.columns if "X" in col and "interaction" not in col]
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = create_cv(train_df, best_features)
for i in range(default_cv):
    X_train_arr[i] = float64_to_float32(X_train_arr[i])
    X_test_arr[i] = float64_to_float32(X_test_arr[i])
    Y_train_arr[i] = float64_to_float32(Y_train_arr[i])
    Y_test_arr[i] = float64_to_float32(Y_test_arr[i])
X_train_arr, X_test_arr, Y_train_arr, Y_test_arr = normal_cv_to_mlx_cv(X_train_arr, X_test_arr, Y_train_arr, Y_test_arr)
best_features_good = ['X757', 'X758', 'X759', 'X508', 'X614', 'X752', 'X331', 'X445', 'X465', 'X385', 
                      'X466', 'X95', 'X23', 'X219', 'X31', 'X373', 'X379', 'X284', 'X750', 'X652', 
                      'X279', 'X89', 'X169', 'X753', 'X226', 'X28', 'X444', 'X272', 'X271', 'X218']
X_train_good_arr, X_test_good_arr, _, _ = create_cv(train_df, best_features_good)
for i in range(default_cv):
    X_train_good_arr[i] = float64_to_float32(X_train_good_arr[i])
    X_test_good_arr[i] = float64_to_float32(X_test_good_arr[i])
X_train_good_arr, X_test_good_arr, _, _ = normal_cv_to_mlx_cv(X_train_good_arr, X_test_good_arr)

In [ ]:
ae_default_num_layers = 2
mlp_default_num_layers = 1

In [ ]:
optimize_aemlp_mlx(
    f"aemlp_mlx_{feature_version}_{default_cv}_{default_random_state}_{ae_default_num_layers}_{mlp_default_num_layers}_with_good_concat_study",
    f"aemlp_mlx_{feature_version}_{default_cv}_{default_random_state}_{ae_default_num_layers}_{mlp_default_num_layers}_with_good_concat_study",
    objective_function = objective_aemlp_mlx_with_concat
)